In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
import json
import torch

In [2]:
import dataset
import algorithm

In [3]:
CONFIG = json.load(open('config.json'))

In [4]:
data = dataset.download_dataset()

Into folder: Dataset/Not/Argetina
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND1.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND2.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND3.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	Segmentize file: CBERS_4A_WPM_20210416_219_155_L2_BAND4.tif
		File Shape: (14879, 14498, 3)
		Images div: 1/537932, 0%
		Images div: 53794/537932, 10%
		Images div: 107587/537932, 20%
		Images div: 161380/537932, 30%
	JoinBands
Into folder: Dataset/Yes/Argentina
	Segmentize file: CBERS_4A_WPM_20210119_224_147_

In [5]:
data = data[:len(data)//10]

In [6]:
print(len(data))

31643


In [7]:
normalized_data = dataset.Normalizer.transform(data)

In [8]:
data_x, data_y = dataset.split_data_x_y(normalized_data)

In [9]:
print(data_y)

[0 1 0 ... 0 0 1]


In [10]:
data_x_train, data_x_test = dataset.split_data_train_test(data_x, CONFIG["data"]["train_split_size"])
data_y_train, data_y_test = dataset.split_data_train_test(data_y, CONFIG["data"]["train_split_size"])

In [11]:
data_x_train.shape

(25314, 20, 20, 4, 3)

In [12]:
dataset_train = dataset.TimeSeriesDataset(data_x_train, data_y_train)
dataset_test = dataset.TimeSeriesDataset(data_x_test, data_y_test)

In [13]:
train_dataloader = DataLoader(dataset_train, CONFIG["training"]["batch_size"], shuffle=True)
test_dataloader = DataLoader(dataset_test, CONFIG["training"]["batch_size"], shuffle=True)

In [14]:
convnet = algorithm.CNNModel(config_model=CONFIG["model"], config_training=CONFIG["training"])

In [15]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(convnet.parameters(), lr=0.001, momentum=0.9)

In [16]:
# Entrenamiento del modelo
for epoch in range(CONFIG["training"]["num_epoch"]):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_dataloader):
        # get the inputs; data is a list of [inputs, labels]
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = convnet(inputs)
        loss = criterion(outputs, labels.type(torch.LongTensor))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

c:\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,  2000] loss: 0.693
[1,  4000] loss: 0.692
[1,  6000] loss: 0.689
[2,  2000] loss: 0.670
[2,  4000] loss: 0.623
[2,  6000] loss: 0.575
[3,  2000] loss: 0.572
[3,  4000] loss: 0.565
[3,  6000] loss: 0.563
[4,  2000] loss: 0.549
[4,  4000] loss: 0.555
[4,  6000] loss: 0.553
[5,  2000] loss: 0.550
[5,  4000] loss: 0.551
[5,  6000] loss: 0.541
[6,  2000] loss: 0.542
[6,  4000] loss: 0.541
[6,  6000] loss: 0.543
[7,  2000] loss: 0.540
[7,  4000] loss: 0.536
[7,  6000] loss: 0.539
[8,  2000] loss: 0.552
[8,  4000] loss: 0.534
[8,  6000] loss: 0.537
[9,  2000] loss: 0.529
[9,  4000] loss: 0.536
[9,  6000] loss: 0.538
[10,  2000] loss: 0.529
[10,  4000] loss: 0.537
[10,  6000] loss: 0.520
Finished Training


In [23]:
correct = 0
total = 0
with torch.no_grad():
    for idx, (images, labels) in enumerate(test_dataloader):
        outputs = convnet(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {0} test images: {1} %'.format(total, 100*correct/total))

Accuracy of the network on the 2110 test images: 51.13744075829384 %


In [26]:
for idx, (x, y) in enumerate(test_dataloader):
    out_batch = convnet(x)
#     print(out_batch)
    _, predicted = torch.max(out_batch, 1)
    predicted = predicted.numpy()
    label = y.numpy()
#     print(y)
    for i in range(len(label)):
        print('Label predict: {0}, Label real: {1}'.format(predicted[i], label[i]))
    break

Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 0
Label predict: 0, Label real: 1
Label pr